第0步：环境准备 (Setup)

操作：在你的终端或CMD中，或者直接在Jupyter Notebook的一个代码块里运行以下命令（去掉前面的!）。

In [ ]:
!pip install PyMuPDF openai tqdm pandas scikit-learn

第1步：导入所有需要的工具包 (Import Libraries)

In [ ]:
import os
import fitz  # PyMuPDF库，用于读取PDF
import openai # OpenAI官方库，用于调用GPT模型
from tqdm.notebook import tqdm # 一个漂亮的进度条库，适用于Notebook
import pandas as pd # 强大的数据处理库，我们用它来处理和分割数据
from sklearn.model_selection import train_test_split # 一个机器学习工具，可以帮我们轻松分割数据集
import json
import re

print("✅ 所有工具包已成功导入！")

第2步：【！！！核心配置区！！！】

In [ ]:
# =====================================================================================
# 2. 【！！！核心配置区！！！】
# =====================================================================================
# 是什么：这里是你唯一需要手动修改的地方！请根据你的实际情况填写下面的参数。

# --- 2.1: 配置你要使用的AI模型服务 ---
# 说明：我们提供多种预设，你可以选择一个，并填入你的信息。
#       将你不需要的配置块用三个引号 """ 注释掉。

# ---【选项A：官方OpenAI】---
AI_PROVIDER = "openai"
OPENAI_API_KEY = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # <--- 在这里填入你的OpenAI API Key
OPENAI_BASE_URL = "https://api.openai.com/v1" # 官方地址，通常无需修改
MODEL_NAME = "gpt-4-turbo-preview" # 使用的模型名称

"""
# ---【选项B：国内服务商/自定义API (示例)】---
# 适用于Kimi (Moonshot AI), 阿里通义千问, Baichuan, ZhipuAI等提供了兼容OpenAI接口的服务
AI_PROVIDER = "custom"
CUSTOM_API_KEY = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx" # <--- 填入你的服务商提供的Key
CUSTOM_BASE_URL = "https://api.moonshot.cn/v1" # <--- 【重要】填入服务商的API地址
MODEL_NAME = "moonshot-v1-8k" # <--- 【重要】填入服务商指定的模型名称
"""

# --- 2.2: 指定你要处理的PDF文件或文件夹 ---
# 说明：三种模式任选其一，将你不需要的模式用#号注释掉即可。

# 模式A：处理单个PDF文件
pdf_paths = ["./中枢性性早熟诊断与治疗专家共识(2022).pdf"]

# # 模式B：处理多个指定的PDF文件
# pdf_paths = [
#     "./医学文档1.pdf",
#     "./医学文档2.pdf",
# ]

# # 模式C：处理一个文件夹中所有的PDF文件
# pdf_folder = "./My_PDFs/" # <--- 指定你的PDF文件夹路径
# pdf_paths = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf")]


# --- 2.3: 指定输出文件的路径 ---
output_train_file = "./train_from_pdf.jsonl"
output_val_file = "./val_from_pdf.jsonl"

# --- 2.4: 配置数据分割比例 ---
VALIDATION_SET_RATIO = 0.1

# =====================================================================================
# --- 配置自动加载 ---
# 下面的代码会根据你上面的选择，自动创建API客户端，无需修改
# =====================================================================================
try:
    if AI_PROVIDER == "openai":
        client = openai.OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_BASE_URL)
        print(f"✅ 已配置使用官方OpenAI服务。模型: {MODEL_NAME}")
    elif AI_PROVIDER == "custom":
        client = openai.OpenAI(api_key=CUSTOM_API_KEY, base_url=CUSTOM_BASE_URL)
        print(f"✅ 已配置使用自定义API服务。模型: {MODEL_NAME}, API地址: {CUSTOM_BASE_URL}")
    else:
        raise ValueError("无效的AI_PROVIDER选项，请选择 'openai' 或 'custom'")
except NameError:
    raise NameError("请在上面的配置区选择一个AI服务商并填入信息！")

print("--- 配置检查 ---")
print(f"待处理的PDF文件数量: {len(pdf_paths)}")
print("PDF文件列表:")
for p in pdf_paths:
    print(f"  - {p}")
print(f"训练集输出路径: {output_train_file}")
print(f"验证集输出路径: {output_val_file}")
print("--- 配置检查完毕 ---")

第3步：定义核心功能函数 (Define Core Functions)

In [ ]:
# =====================================================================================
# 第3步：定义核心功能函数 (Define Core Functions)
# =====================================================================================
# 说明：这里我们把整个流程中需要用到的功能，封装成一个个独立的“小工具”（函数）。
#       你不需要修改这部分代码。

def extract_text_from_pdfs(pdf_paths):
    """
    函数功能：从一个或多个PDF文件中提取所有文本。
    """
    full_text = ""
    print("--- 开始提取PDF文本 ---")
    for pdf_path in tqdm(pdf_paths, desc="提取PDF进度"):
        try:
            doc = fitz.open(pdf_path)
            full_text += f"\n\n===== Start of Document: {os.path.basename(pdf_path)} =====\n\n"
            for page in doc:
                full_text += page.get_text("text")
            full_text += f"\n\n===== End of Document: {os.path.basename(pdf_path)} =====\n\n"
        except Exception as e:
            print(f"⚠️警告：读取文件 {pdf_path} 时出错: {e}")
    print(f"✅ 所有PDF文本提取完成，总计 {len(full_text)} 个字符。")
    return full_text

def chunk_text(text, min_chunk_size=300, max_chunk_size=700):
    """
    函数功能：将长文本智能地切分成多个小块。
    """
    print("\n--- 开始智能文本分块 ---")
    paragraphs = [p.strip() for p in text.split('\n\n') if len(p.strip()) > 10]
    chunks, final_chunks, temp_chunk = [], [], ""
    for para in paragraphs:
        if len(para) <= max_chunk_size: chunks.append(para)
        else:
            sentences = re.split(r'(?<=[。？！])\s*', para)
            current_chunk = ""
            for sentence in sentences:
                if len(current_chunk) + len(sentence) <= max_chunk_size: current_chunk += sentence
                else: chunks.append(current_chunk); current_chunk = sentence
            if current_chunk: chunks.append(current_chunk)
    for chunk in chunks:
        if len(temp_chunk) < min_chunk_size: temp_chunk += "\n" + chunk
        else: final_chunks.append(temp_chunk); temp_chunk = chunk
    if temp_chunk: final_chunks.append(temp_chunk)
    print(f"✅ 文本被成功切分为 {len(final_chunks)} 个小块。")
    return final_chunks

def generate_qa_from_chunk(text_chunk, model_name):
    """
    函数功能：调用已配置好的大模型API客户端，从单个文本块生成Q&A。
    """
    prompt_template = f"""
# 角色
你是一位专业的医学知识库构建专家。

# 任务
你的任务是根据下面提供的【原始文本】，生成3到5个高质量的、符合JSONL格式的问答(Q&A)对。

# 要求
1. **忠于原文**：所有答案都必须完全基于【原始文本】的内容，不允许捏造或推理【原始文本】中没有的信息。
2. **一问一答**：每个问题都应该有一个简洁、完整、独立的答案。
3. **多样性**：问题应该从不同角度提出，涵盖定义、原因、诊断标准、治疗方法、注意事项等。避免提出答案相似的重复问题。
4. **格式严格**：每个Q&A对必须是独立的JSON对象，格式为 `{{"input": "你的问题", "output": "你的答案"}}`，并且每行一个JSON对象。不要在开头或结尾添加任何额外的解释或代码块标记。

# 【原始文本】
{text_chunk}
"""
    try:
        # 【！！！核心修改！！！】
        # 使用我们在第2步创建的全局变量 `client` 来发送请求。
        # 这样无论你配置的是OpenAI还是自定义服务，这里的代码都无需改变。
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed to output well-formed JSONL."},
                {"role": "user", "content": prompt_template}
            ],
            temperature=0.3,
            max_tokens=1024,
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"❌ API调用失败: {e}")
        return None

# =====================================================================================
# 第4步：执行主流程 (Run the Main Process)
# =====================================================================================
# 说明：现在，我们将上面定义好的所有“小工具”按顺序串联起来，执行完整的处理流程。
# 操作：直接运行下面的代码块即可。

# --- 流程1: 提取文本 ---
full_text = extract_text_from_pdfs(pdf_paths)

# --- 流程2: 文本分块 ---
text_chunks = chunk_text(full_text)

# --- 流程3: AI生成Q&A ---
print("\n--- 开始调用AI批量生成Q&A (这可能需要一些时间) ---")
all_qa_pairs = []
for chunk in tqdm(text_chunks, desc="AI生成进度"):
    # 【！！！核心修改！！！】将全局配置的MODEL_NAME传入函数
    qa_pairs_str = generate_qa_from_chunk(chunk, MODEL_NAME)
    if qa_pairs_str:
        for line in qa_pairs_str.strip().split("\n"):
            try:
                qa_pair = json.loads(line)
                if "input" in qa_pair and "output" in qa_pair:
                    all_qa_pairs.append(qa_pair)
            except json.JSONDecodeError:
                print(f"⚠️警告：无法解析AI返回的一行内容: {line}")

print(f"\n✅ 成功生成了 {len(all_qa_pairs)} 条Q&A数据！")

# --- 流程4: 数据清洗和分割 ---
if all_qa_pairs:
    print("\n--- 开始数据清洗和分割 ---")
    df = pd.DataFrame(all_qa_pairs)
    df.dropna(inplace=True)
    df.drop_duplicates(subset=["input"], inplace=True)
    
    train_df, val_df = train_test_split(df, test_size=VALIDATION_SET_RATIO, random_state=42)
    
    train_df.to_json(output_train_file, orient='records', lines=True, force_ascii=False)
    val_df.to_json(output_val_file, orient='records', lines=True, force_ascii=False)
    
    print(f"✅ 数据处理完成！")
    print(f"   - 训练集 ({len(train_df)}条) 已保存到: {output_train_file}")
    print(f"   - 验证集 ({len(val_df)}条) 已保存到: {output_val_file}")
else:
    print("❌ 未能生成任何Q&A数据，请检查API配置或PDF内容。")